# Bandit Problem

In [1]:
# imports
from bandits import Bandit
import random
# Include your imports here, if any are used. 

A bandit is one option (or “arm”) you can choose, where the reward you get is uncertain and must be learned by trying it out.
In multi-armed bandits, you repeatedly pick among several such uncertain options to find which one pays best.

A list of ten bandit objects initialized in the list...

In [2]:
bandits = [Bandit(random.random()*4-2) for _ in range(10)]

To generate reward from that bandit, use the pullLever() command

In [3]:
bandits[0].pullLever()

0.04364432528747145

## Greedy algorithm Implementation

In [4]:
def run_greedy(no_of_iterations):
  avg_till_now=[0 for _ in range(10)]
  reward_list=[]
  for iterations in range(1,no_of_iterations+1):
      reward=0
      maximum=max(avg_till_now)
      max_avg=[]
      for j in range(10):
          if avg_till_now[j]==maximum:
              max_avg.append(j)
      if len(max_avg)==1:
          reward = bandits[max_avg[0]].pullLever()
          reward_list.append(reward)
          avg_till_now[max_avg[0]]+=(1/iterations) * (reward-avg_till_now[max_avg[0]])
      
      if len(max_avg)!=1:
          selected=random.choice(max_avg)
          reward = bandits[selected].pullLever()
          reward_list.append(reward)
          avg_till_now[selected]+=(1/iterations) * (reward-avg_till_now[selected])
  return reward_list


Plot the cumulative average of rewards as the number of iterations increases. and display that image below.

## $\epsilon$-greedy Algorithm

In [ ]:
def run_epsilon_greedy(epsilon):
    reward_list = []
    reward=0
    avg=[0]*10
    is_epsilion = [True, False]
    probablities = [epsilon, 1-epsilon]
    times_chosen=[0]*10
    for i in range(1,1001):
        if(np.random.choice(is_epsilion, p=probablities)):
            e_pull = random.randint(0,9)
            reward = bandits[e_pull].pullLever()
            times_chosen[e_pull]+=1
            reward_list.append(reward)
            avg[e_pull]+=1/times_chosen[e_pull] * (reward-avg[e_pull])
        else:
            greedy=max(avg)
            greedy_list=[]
            for j in range(10):
                if avg[j]==greedy:
                    greedy_list.append(j)
            if len(greedy_list)==1:
                reward=bandits[greedy_list[0]].pullLever()
                reward_list.append(reward)
                times_chosen[greedy_list[0]]+=1
                avg[greedy_list[0]]+=1/times_chosen[greedy_list[0]] * (reward-avg[greedy_list[0]])
            else:
                select=random.choice(greedy_list)
                reward=bandits[select].pullLever()
                
                times_chosen[select]+=1
                avg[select]+=1/times_chosen[select] * (reward-avg[select])
    return reward_list

Plot the cumulative average of rewards as the number of iterations increases but for various values of $\epsilon$.

## Finding the optimal $\epsilon$

Run the $\epsilon$-greedy algorithm for 1000 iterations and find the optimal $\epsilon$ value by plotting the cumulative average of rewards for various values of $\epsilon$

## Optimistic Initial Values

In [6]:
def run_optimistic_greedy():
    # TODO: Implement the optimistic greedy algorithm here

    # Return the reward from the bandits in a list
    pass

Plot the cumulative average of rewards as the number of iterations increases for an optimistic greedy of $Q_1 = 10$ and a non-optimistic $\epsilon = 0.1$ and try to compare which is better.

## Upper Confidence Bound (UCB)

In [7]:
def run_ucb(c):
    # TODO: Implement the UCB algorithm here
    # Return the reward from the bandits in a list
    pass